1. Считать в `pandas.DataFrame` любой источник данных: CSV, JSON, Excel-файл, HTML-таблицу и т.п.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


df = pd.read_csv('games.csv')

print(df.head())
print(df.info())

         id  rated    created_at  last_move_at  turns victory_status winner  \
0  TZJHLljE  False  1.504210e+12  1.504210e+12     13      outoftime  white   
1  l1NXvwaE   True  1.504130e+12  1.504130e+12     16         resign  black   
2  mIICvQHh   True  1.504130e+12  1.504130e+12     61           mate  white   
3  kWKvrqYL   True  1.504110e+12  1.504110e+12     61           mate  white   
4  9tXo1AUZ   True  1.504030e+12  1.504030e+12     95           mate  white   

  increment_code       white_id  white_rating      black_id  black_rating  \
0           15+2       bourgris          1500          a-00          1191   
1           5+10           a-00          1322     skinnerua          1261   
2           5+10         ischia          1496          a-00          1500   
3           20+0  daniamurashov          1439  adivanov2009          1454   
4           30+3      nik221107          1523  adivanov2009          1469   

                                               moves opening_e

2. Датасет и подготовка данных:
   - Привести описание датасета.
   - Осуществить предобработку данных (избавиться от `null`, убрать некоторые признаки и т.п.) - "подчистить данные".
   - Закодировать категориальные признаки при необходимости.
   - Нормализовать данные.
   - Разбить выборку на обучающую и тестовую.
     > Далее используем обучающую выборку, в том числе для метрик.

Цель: предсказание победителя шахматной партии (winner) на основе метаданных игры.
Признаки:
- rated: является ли игра рейтинговой
- created_at / last_move_at: временные метки начала и окончания
- turns: количество ходов
- victory_status: способ завершения игры
- increment_code: контроль времени
- white_id, black_id: идентификаторы игроков
- white_rating, black_rating: рейтинги игроков
- moves: запись ходов
- opening_eco, opening_name, opening_ply: информация о дебюте

In [ ]:
df.drop(['id', 'created_at', 'last_move_at', 'white_id', 'black_id', 'moves'], axis=1, inplace=True)

print("\nКоличество пропусков:")
print(df.isnull().sum())
df.dropna(inplace=True)

label_encoder = LabelEncoder()
df['winner'] = label_encoder.fit_transform(df['winner'])

df['rated'] = df['rated'].map({False: 0, True: 1})

categorical_cols = ['victory_status', 'increment_code', 'opening_eco', 'opening_name']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

scaler = StandardScaler()
numeric_cols = ['turns', 'white_rating', 'black_rating', 'opening_ply']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

X = df.drop('winner', axis=1)
y = df['winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nРазмер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

def evaluate_model(model, name, X_train, y_train):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)

    acc = accuracy_score(y_train, y_pred)
    prec = precision_score(y_train, y_pred, average='weighted')
    rec = recall_score(y_train, y_pred, average='weighted')
    f1 = f1_score(y_train, y_pred, average='weighted')

    print(f"\n{name} на обучающей выборке:")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-score: {f1:.4f}")
    return acc, prec, rec, f1


Количество пропусков:
rated             0
turns             0
victory_status    0
winner            0
increment_code    0
white_rating      0
black_rating      0
opening_eco       0
opening_name      0
opening_ply       0
dtype: int64

Размер обучающей выборки: (16046, 2247)
Размер тестовой выборки: (4012, 2247)


3. Дерево решений:
   - С использованием `GridSearchCV` осуществить подбор гиперпараметра `DecisionTreeClassifier` (как минимум `max_depth`, `max_features`, другие параметры - по желанию.)
   - Вывести значения гиперпараметра и метрик для наилучшей модели `DecisionTreeClassifier` ($accuracy$, $precision$, $recall$, $\textit{f-measure}$).
   - Для полученного наилучшего дерева вывести `feature_importances`, отсортировать их по убыванию.
   - Осуществить фильтрацию признаков (по какому-нибудь значению порога важности признака).
   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.
   - Вывести полученные гиперпараметры лучшей модели.
   - Сравнить метрики до и после фильтрации признаков лучших моделей

In [ ]:
param_grid_tree = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

grid_tree = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_tree, cv=5, scoring='accuracy', n_jobs=-1)
grid_tree.fit(X_train, y_train)

best_tree = grid_tree.best_estimator_
print(f"\nЛучшие параметры дерева: {grid_tree.best_params_}")

evaluate_model(best_tree, "Decision Tree", X_train, y_train)

feature_importances = pd.Series(best_tree.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nFeature Importances:")
print(feature_importances.head(10))

threshold = 0.01
selected_features = feature_importances[feature_importances > threshold].index.tolist()
print(f"\nВыбранные признаки после фильтрации (порог >{threshold}):")
print(selected_features)

X_train_filtered = X_train[selected_features]
X_test_filtered = X_test[selected_features]

grid_tree_filtered = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_tree, cv=5, scoring='accuracy', n_jobs=-1)
grid_tree_filtered.fit(X_train_filtered, y_train)

best_tree_filtered = grid_tree_filtered.best_estimator_
print(f"\nЛучшие параметры дерева (на отфильтрованных признаках): {grid_tree_filtered.best_params_}")

print("\n--- Сравнение деревьев ---")
print("До фильтрации:")
evaluate_model(best_tree, "Decision Tree (до)", X_train, y_train)
print("После фильтрации:")
evaluate_model(best_tree_filtered, "Decision Tree (после)", X_train_filtered, y_train)



Лучшие параметры дерева: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2}

Decision Tree на обучающей выборке:
Accuracy: 0.5032, Precision: 0.6386, Recall: 0.5032, F1-score: 0.3448

Feature Importances:
opening_ply                          0.079634
black_rating                         0.078023
opening_eco_C41                      0.060900
opening_eco_A04                      0.056965
increment_code_5+30                  0.055840
increment_code_20+60                 0.052933
turns                                0.052357
opening_name_Caro-Kann Defense #2    0.039176
white_rating                         0.038516
increment_code_15+15                 0.035481
dtype: float64

Выбранные признаки после фильтрации (порог >0.01):
['opening_ply', 'black_rating', 'opening_eco_C41', 'opening_eco_A04', 'increment_code_5+30', 'increment_code_20+60', 'turns', 'opening_name_Caro-Kann Defense #2', 'white_rating', 'increment_code_15+15', 'increment_code_30+0', 'o

(0.5929826748099215,
 0.5936280960002484,
 0.5929826748099215,
 0.5844947897105054)

4. Случайный лес
   - Построить случайный лес (`RandomForestClassifier`), c использованием `GridSearchCV` осуществить подбор гиперпараметра.
   - Вывести полученные гиперпараметры лучшей модели случайного леса.
   - Осуществить фильтрацию признаков.
   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.
   - Вывести полученные гиперпараметры лучшей модели случайного леса.
   - Сравнить метрики до и после фильтрации признаков лучших моделей.

In [ ]:
param_grid_forest = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

grid_forest = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_forest, cv=5, scoring='accuracy', n_jobs=-1)
grid_forest.fit(X_train, y_train)

best_forest = grid_forest.best_estimator_
print(f"\nЛучшие параметры случайного леса: {grid_forest.best_params_}")
evaluate_model(best_forest, "Random Forest", X_train, y_train)

X_train_filtered = X_train[selected_features]
X_test_filtered = X_test[selected_features]

grid_forest_filtered = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_forest, cv=5, scoring='accuracy', n_jobs=-1)
grid_forest_filtered.fit(X_train_filtered, y_train)

best_forest_filtered = grid_forest_filtered.best_estimator_
print(f"\nЛучшие параметры случайного леса (на отфильтрованных признаках): {grid_forest_filtered.best_params_}")

print("\n--- Сравнение случайного леса ---")
print("До фильтрации:")
evaluate_model(best_forest, "Random Forest (до)", X_train, y_train)
print("После фильтрации:")
evaluate_model(best_forest_filtered, "Random Forest (после)", X_train_filtered, y_train)



Лучшие параметры случайного леса: {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 200}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Random Forest на обучающей выборке:
Accuracy: 0.5328, Precision: 0.6537, Recall: 0.5328, F1-score: 0.4093

Лучшие параметры случайного леса (на отфильтрованных признаках): {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 200}

--- Сравнение случайного леса ---
До фильтрации:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Random Forest (до) на обучающей выборке:
Accuracy: 0.5328, Precision: 0.6537, Recall: 0.5328, F1-score: 0.4093
После фильтрации:

Random Forest (после) на обучающей выборке:
Accuracy: 0.6584, Precision: 0.6368, Recall: 0.6584, F1-score: 0.6348


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.6584195438115418,
 0.6368017463123653,
 0.6584195438115418,
 0.6348413559457295)

5. Метод ближайших соседей:
   - С использованием `GridSearchCV` осуществить подбор гиперпараметра `KNeighborsClassifier` (`n_neighbors`).
   - Вывести значения гиперпараметра и метрик для наилучшей модели.
   - Осуществить фильтрацию признаков.
   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.
   - Вывести полученные гиперпараметры лучшей модели случайного леса.
   - Сравнить метрики до и после фильтрации признаков.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_knn_simple = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform'],
    'p': [2]
}

grid_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_knn_simple,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

grid_knn.fit(X_train, y_train)

best_knn = grid_knn.best_estimator_
print(f"\nЛучшие параметры KNN: {grid_knn.best_params_}")

evaluate_model(best_knn, "KNN", X_train, y_train)

X_train_filtered = X_train[selected_features]
X_test_filtered = X_test[selected_features]

grid_knn_filtered = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_knn_simple,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)
grid_knn_filtered.fit(X_train_filtered, y_train)

best_knn_filtered = grid_knn_filtered.best_estimator_
print(f"\nЛучшие параметры KNN (на отфильтрованных признаках): {grid_knn_filtered.best_params_}")

print("\n--- Сравнение KNN ---")
print("До фильтрации:")
evaluate_model(best_knn, "KNN (до)", X_train, y_train)
print("После фильтрации:")
evaluate_model(best_knn_filtered, "KNN (после)", X_train_filtered, y_train)


Лучшие параметры KNN: {'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}

KNN на обучающей выборке:
Accuracy: 0.7023, Precision: 0.7075, Recall: 0.7023, F1-score: 0.6996

Лучшие параметры KNN (на отфильтрованных признаках): {'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}

--- Сравнение KNN ---
До фильтрации:

KNN (до) на обучающей выборке:
Accuracy: 0.7023, Precision: 0.7075, Recall: 0.7023, F1-score: 0.6996
После фильтрации:

KNN (после) на обучающей выборке:
Accuracy: 0.6975, Precision: 0.6946, Recall: 0.6975, F1-score: 0.6937


(0.6974947027296523,
 0.6946446546716438,
 0.6974947027296523,
 0.6936706084029484)

6. Если наблюдается улучшение метрик после фильтрации признаков хотя бы для одной из моделей, то для набора отфильтрованных признаков (пересечение множеств отфильтрованных признаков каждой модели или объединение множеств &ndash; не особо важно, главное описать, каким образом получен новый subset данных) заново построить наилучшие модели `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier`, сравнить модели в пункте 7 на одинаковом полученном наборе отфильтрованных признаков. Иначе &ndash; пропустить этот пункт.

In [ ]:
final_models = {
    "Decision Tree": best_tree_filtered,
    "Random Forest": best_forest_filtered,
    "KNN": best_knn_filtered
}

results = []

for name, model in final_models.items():
    model.fit(X_train_filtered, y_train)
    y_pred = model.predict(X_test_filtered)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results.append((name, acc, prec, rec, f1))
    print(f"\n{name}:")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-score: {f1:.4f}")



Decision Tree:
Accuracy: 0.5503, Precision: 0.5432, Recall: 0.5503, F1-score: 0.5431


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Random Forest:
Accuracy: 0.6129, Precision: 0.5889, Recall: 0.6129, F1-score: 0.5886

KNN:
Accuracy: 0.5698, Precision: 0.5652, Recall: 0.5698, F1-score: 0.5666


7. Оценка качества построенных моделей:
   - Визуализировать любое полученное дерево решений.
     > Для вывода названий признаков в граф необходимо задать значение аргумента `feature_names` в `sklearn.tree.export_graphviz`, для вывода названий классов &ndash; `class_names` (перед кодированием целевого признака можно сохранить названия в отдельный массив).
   - Сравнить лучшие модели `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier` на **тестовой выборке**. Привести значения метрик $accuracy$, $precision$, $recall$, $\textit{f-measure}$.

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

class_names = label_encoder.classes_.tolist()
feature_names = X_train_filtered.columns.tolist()

dot_data = export_graphviz(
    best_tree_filtered,
    out_file=None,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    rounded=True,
    special_characters=True
)

graph = graphviz.Source(dot_data)
graph.render("decision_tree")
graph.view()

'decision_tree.pdf'